<a href="https://colab.research.google.com/github/MunJinSeo/MachineLearning/blob/main/%EA%B8%B0%EA%B3%84%ED%95%99%EC%8A%B5(AAI107)_Team13_%EB%89%B4%EC%8A%A4%ED%81%AC%EB%A1%A4%EB%A7%81_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D_20220608(%ED%9A%8C%EC%82%AC2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1) 증권 뉴스 크롤링

- 팍스넷이나 네이버 증권 뉴스는 최근 뉴스를 가져오기 수월하지만
과거 특정종목에 대해 검색이 어렵고 2~3년 지난 뉴스자료가 없는 경우도 있음
- 따라서 빅카인즈 https://www.bigkinds.or.kr/  사이트를 활용하여 
삼성전자 관련 뉴스를 검색하여 감성분석에 사용하기로함
 (빅카인즈 뉴스 자료는 해당 사이트에서 수작업으로 추출했으며, 해당 파일을 나의 github에 올려 놓은 후 아래 감성분석에 사용함)

In [ ]:
# 팍스넷, 네이버 증권 뉴스 크롤링
# https://github.com/CSJ516/Stock_Prediction_project

import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib import parse
from datetime import datetime

In [ ]:
# all_data 대상일자 추출
#target_date =  all_data.index.strftime('%Y%m%d').copy() # pd.DataFrame(all_data.index).copy()
target_date = {"Date":['20220601','20220602','20220603']}
df_target_date = pd.DataFrame(target_date)
print(df_target_date)
print(type(df_target_date))
print(df_target_date["Date"].tail(1))

       Date
0  20220601
1  20220602
2  20220603
<class 'pandas.core.frame.DataFrame'>
2    20220603
Name: Date, dtype: object


### 팍스넷 > 뉴스 > 많이본뉴스 > 증권

In [ ]:
def paxnet_news_title(date_list):
  # 팍스넷 > 뉴스 > 많이본뉴스 > 증권
  paxnet_url = 'http://www.paxnet.co.kr/news/much?newsSetId=4667&currentPageNo={}&genDate={}&objId=N4667'
  #paxnet_url = 'http://www.paxnet.co.kr/news/much?newsSetId=4667&objId=N4667'
  
  webheader = {
      'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
  }

  paxnet_result = []
  paxnet_error_cnt = 0
  
  for t_date in date_list:
    for pagecnt in range(1, 2):
      target_url = paxnet_url.format(pagecnt, t_date)
      print(target_url)
      responseData = requests.get(target_url, headers=webheader)
      if responseData.status_code == 200:
        resParse = BeautifulSoup(responseData.text)
        title_list = resParse.select('ul.thumb-list li')
        for ttt in title_list:
          #print(ttt)
          try:
            news_title = ttt.select_one('dl.text > dt').text.strip()
            paxnet_result.append([news_title])
          except:
            paxnet_error_cnt += 1
  return paxnet_result

In [ ]:
paxnet_news_rs = paxnet_news_title(df_target_date["Date"].tail(1)) # 샘플로 최근 1일자만 추출해봄
#paxnet_news_rs = paxnet_news_title(df_target_date["Date"].head(1)) # 샘플로 최초 1일자만 추출해봄
print("paxnet_news_rs length == ", len( paxnet_news_rs ) )

http://www.paxnet.co.kr/news/much?newsSetId=4667&currentPageNo=1&genDate=20220603&objId=N4667
paxnet_news_rs length ==  20


In [ ]:
paxnet_title_df = pd.DataFrame(paxnet_news_rs, columns=['뉴스제목'])
#paxnet_title_df['주가변동'] = 0  # all_data["Class"] 값으로 맵핑 필요 (해당 일자에 맞춰 주가 상승, 보합, 하락에 따라 분류)
paxnet_title_df

,뉴스제목
0,국내 환율 마감시황
1,FTSE100(영국) : Platinum Jubilee Bank Holiday로 휴장
2,국내 채권금리 동향(마감)
3,항셍지수(홍콩) : 단오절(Tuen Ng Festival)로 휴장
4,뉴욕증시 마감시황
5,미국 주요기업 주가등락률(마감)
6,미국 주요 ETF 등락률
7,미국시장 테마시황
8,[뉴욕증시] 테슬라·메타 급등에 일제히 상승…나스닥 2.69% ↑
9,국내외 주요 마감지수


###네이버 > 금융 > 많이본뉴스

In [ ]:
def naver_news_title(date_list):
  # 네이버 > 금융 > 많이본뉴스
  naver_url = 'https://finance.naver.com/news/news_list.naver?mode=RANK&page={}&date={}'
  
  webheader = {
      'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
  }

  naver_result = []
  naver_error_cnt = 0

  for t_date in date_list:
    for pagecnt in range(1, 2):
      target_url = naver_url.format(pagecnt, t_date)
      print(target_url)
      responseData = requests.get(target_url, headers=webheader)
      if responseData.status_code == 200:
        resParse = BeautifulSoup(responseData.text)
        title_list = resParse.select('ul.simpleNewsList li')
        for ttt in title_list:
          #print("-----------------\n")
          #print(ttt)
          try:
            #news_title = ttt.select_one('li').text.strip()
            news_title = ttt.select_one('a').text.strip()
            #print("news_title==",news_title)
            naver_result.append([news_title])
          except:
            naver_error_cnt += 1
  return naver_result

In [ ]:
naver_news_rs = naver_news_title(df_target_date["Date"].tail(1)) # 샘플로 최근 1일자만 추출해봄
print("naver_news_rs length == ", len( naver_news_rs ) )

https://finance.naver.com/news/news_list.naver?mode=RANK&page=1&date=20220603
naver_news_rs length ==  25


In [ ]:
naver_title_df = pd.DataFrame(naver_news_rs, columns=['뉴스제목'])
#naver_title_df['주가변동'] = 0  # all_data["Class"] 값으로 맵핑 필요 (해당 일자에 맞춰 주가 상승, 보합, 하락에 따라 분류)
naver_title_df

,뉴스제목
0,"델리오, 첫 콜라보 NFT ‘클레이다이노 블루베이 에디션’ 경매.."
1,"[특징주] 한국조선해양, 수주 계약 취소에도… 주가는 2% 상승"
2,"거래소, ITX-AI 상장적격성 실질심사 사유 추가"
3,[특징주] 인천공항 국제선 정상화에 항공주 강세
4,[특징주] 우크라 외교차관 방한에 재건주 '강세'… 서암기계공업..
5,"[특징주] 청담글로벌, 코스닥 상장 첫날 공모가 대비 27%↑(.."
6,"금융위, 중기 특화 증권사 7곳 지정…케이프·DS 포함"
7,"에스티아이, 잉크젯 장비 수주 계약 체결"
8,"[특징주]디와이파워, 재건 중장비 '근육' 담당…우크라이나 재건.."
9,"상승장서 소외된 건설주, 선거 뒤 탄력받을까…증권가는 ""사라"""


### 네이버 > 금융 > 뉴스포커스(시황/전망)

In [ ]:
def naver_market_news_title(date_list):
  # 네이버 > 금융 > 뉴스포커스(시황/전망)
  naver_market_url = 'https://finance.naver.com/news/news_list.naver?mode=LSS3D&section_id=101&section_id2=258&section_id3=401&page={}&date={}'
  
  webheader = {
      'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
  }

  naver_market_result = []
  naver_market_error_cnt = 0

  for t_date in date_list:
    for pagecnt in range(1, 2):
      target_url = naver_market_url.format(pagecnt, t_date)
      print(target_url)
      responseData = requests.get(target_url, headers=webheader)
      if responseData.status_code == 200:
        resParse = BeautifulSoup(responseData.text)
        title_list = resParse.select('dd.articleSubject')
        for ttt in title_list:
          #print("-----------------\n")
          #print(ttt)
          try:
            #news_title = ttt.select_one('li').text.strip()
            news_title = ttt.select_one('a').text.strip()
            #print("news_title==",news_title)
            naver_market_result.append([news_title])
          except:
            naver_market_error_cnt += 1
  return naver_market_result

In [ ]:
naver_market_news_rs = naver_market_news_title(df_target_date["Date"].tail(1)) # 샘플로 최근 1일자만 추출해봄
print("naver_market_news_rs length == ", len( naver_market_news_rs ) )

https://finance.naver.com/news/news_list.naver?mode=LSS3D&section_id=101&section_id2=258&section_id3=401&page=1&date=20220603
naver_market_news_rs length ==  18


In [ ]:
naver_market_title_df = pd.DataFrame(naver_market_news_rs, columns=['뉴스제목'])
#naver_market_title_df['주가변동'] = 0  # all_data["Class"] 값으로 맵핑 필요 (해당 일자에 맞춰 주가 상승, 보합, 하락에 따라 분류)
naver_market_title_df

,뉴스제목
0,"한국투자증권, 미국 부동산 투자자문 서비스 개시"
1,[표]유가증권·코스닥 투자주체별 매매동향(5월 31일-최종치)
2,'목표가 93만원→48만원'…삼성SDI 혹평한 씨티증권에 '소나기 반박'
3,[표] 주간 수익률 베스트&워스트
4,[표] 주요 증시 지표 (6월 3일)
5,[ETF줌인] 명품 투자 ETF…면세점 매출 회복 수혜 기대
6,"대신증권, '비상장주식 안심거래 서비스' 실시"
7,[장외주식] 쓰리디팩토리 나흘 연속 상승
8,[표]코스피200지수 옵션 시세(6월 3일)
9,[표]코스피200지수·국채·달러 선물 시세(6월 3일)


## 2) 금융 뉴스 감정 분석 : 삼성전자 뉴스
- 사용 모델: snunlp/KR-FinBert-SC
- https://huggingface.co/snunlp/KR-FinBert-SC
- https://github.com/snunlp/KR-FinBert-SC

### 기본 환경 설치

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 31.8 MB/s 
     |████████████████████████████████| 596 kB 21.7 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 
     |████████████████████████████████| 6.6 MB 62.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification # pre-trained된 모델 로딩
import torch

import urllib.request # 데이터 파일 URL다운로드
from tqdm import tqdm #모델 예측시 계산 진행상태를 보여줌

#from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset # 데이터셋을 분할하여 사용

import gc #GPU 캐쉬 초기화

In [ ]:
print(torch.cuda.is_available() ) # GPU가용성 확인
print(torch.cuda.current_device() ) #현재 선택한 장치의 인덱스 
print(torch.cuda.device_count() ) #사용할 수 있는 GPU 수
print(torch.cuda.get_device_name(0) ) #장치 이름

True
0
1
Tesla T4


In [ ]:
# GPU or CPU
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
  device = torch.device("cpu")
  print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# GPU 캐쉬 비워줌
gc.collect()
torch.cuda.empty_cache()

### 데이터 준비: 금융 감성분석 샘플 - 모델 초기 확인용

In [ ]:
class Dataset_finance_emoton_csv(Dataset):
  def __init__(self, file_name):
    # --- 전처리 1 start ------------------------
    urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv", filename="finance_emoton_data.csv")
    self.dataset = pd.read_csv('finance_emoton_data.csv')
    #self.dataset = pd.read_csv(file_name, sep=',')

    # tokenizer
    self.tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
    
    # --- 전처리 1 end ------------------------
  
  def __len__(self):
    return len(self.dataset)
  
  def clean_text(self, txt):
    # --- 전처리 2 start ----------------------
    cleaned = self.sub1.sub('', txt.strip())  # .strip()은 문장의 앞뒤 공백제거함
    cleaned = self.sub2.sub(' ', cleaned)
    cleaned = self.sub3.sub('.', cleaned)
    #cleaned = emoticon_normalize(cleaned, num_repeats=3) # 감정 반복 단순화
    #cleaned = repeat_normalize(cleaned, num_repeats=2) # 중복 글자 단순화
    #cleaned = only_text(cleaned) # text만 추출
    #cleaned = only_hangle(cleaned) # 한글만 추출
    #cleaned = only_hangle_number(cleaned) # 한글/숫자만 추출

    #if len(ssStr) > 1:
    cleaned = "[CLS] " + cleaned + " [SEP]"
    # --- 전처리 2 end ------------------------
    return cleaned

  
  def __getitem__(self, idx):

    text = self.dataset['kor_sentence'][idx]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask

### 데이터 준비: 삼성전자 뉴스 2018년~최근

In [ ]:
class Dataset_samsung_news(Dataset):
  def __init__(self, file_name):
    # --- 전처리 1 start ------------------------
    # 자료 출처 : 빅카인즈 https://www.bigkinds.or.kr/
    #  빅카인즈 뉴스 자료는 해당 사이트에서 수작업으로 추출했으며, 해당 파일을 나의 github에 올려 놓은 후 아래 감성분석에 사용함)
    urllib.request.urlretrieve("https://raw.githubusercontent.com/MunJinSeo/MachineLearning/main/samsung_news_20180101-20181231.xlsx", filename="samsung_news_2018.xlsx")
    urllib.request.urlretrieve("https://raw.githubusercontent.com/MunJinSeo/MachineLearning/main/samsung_news_20190101-20191231.xlsx", filename="samsung_news_2019.xlsx")
    urllib.request.urlretrieve("https://raw.githubusercontent.com/MunJinSeo/MachineLearning/main/samsung_news_20200101-20201231.xlsx", filename="samsung_news_2020.xlsx")
    urllib.request.urlretrieve("https://raw.githubusercontent.com/MunJinSeo/MachineLearning/main/samsung_news_20210101-20211231.xlsx", filename="samsung_news_2021.xlsx")
    urllib.request.urlretrieve("https://raw.githubusercontent.com/MunJinSeo/MachineLearning/main/samsung_news_20220101-20220607.xlsx", filename="samsung_news_2022.xlsx")
    
    #self.dataset = pd.read_csv(file_name, sep=',')
    self.dataset = pd.read_excel('samsung_news_2018.xlsx', sheet_name=0, usecols=['일자','언론사','제목','본문'] )
    self.dataset = self.dataset.append(pd.read_excel('samsung_news_2019.xlsx', sheet_name=0, usecols=['일자','언론사','제목','본문'] ), ignore_index = True)
    self.dataset = self.dataset.append(pd.read_excel('samsung_news_2020.xlsx', sheet_name=0, usecols=['일자','언론사','제목','본문'] ), ignore_index = True)
    self.dataset = self.dataset.append(pd.read_excel('samsung_news_2021.xlsx', sheet_name=0, usecols=['일자','언론사','제목','본문'] ), ignore_index = True)
    self.dataset = self.dataset.append(pd.read_excel('samsung_news_2022.xlsx', sheet_name=0, usecols=['일자','언론사','제목','본문'] ), ignore_index = True)

    # tokenizer
    self.tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
    
    # --- 전처리 1 end ------------------------
  
  def __len__(self):
    return len(self.dataset)
  
  def clean_text(self, txt):
    # --- 전처리 2 start ----------------------
    cleaned = self.sub1.sub('', txt.strip())  # .strip()은 문장의 앞뒤 공백제거함
    cleaned = self.sub2.sub(' ', cleaned)
    cleaned = self.sub3.sub('.', cleaned)
    #cleaned = emoticon_normalize(cleaned, num_repeats=3) # 감정 반복 단순화
    #cleaned = repeat_normalize(cleaned, num_repeats=2) # 중복 글자 단순화
    #cleaned = only_text(cleaned) # text만 추출
    #cleaned = only_hangle(cleaned) # 한글만 추출
    #cleaned = only_hangle_number(cleaned) # 한글/숫자만 추출

    #if len(ssStr) > 1:
    cleaned = "[CLS] " + cleaned + " [SEP]"
    # --- 전처리 2 end ------------------------
    return cleaned

  
  def __getitem__(self, idx):

    text = self.dataset['제목'][idx] #제목, 본문 각각 해봐야할듯함

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True, #`pad_to_max_length` 인수는 더 이상 사용되지 않으며 향후 버전에서 제거. 일괄 처리에서 가장 긴 시퀀스로 채우려면 `padding=True` 또는 `padding='longest'`를 사용하거나 `padding='max_length'를 사용하세요. ` 최대 길이로 채웁니다. 이 경우 `max_length`로 특정 길이를 지정하거나(예: `max_length=45`) max_length를 None으로 두어 모델의 최대 입력 크기(예: Bert의 경우 512)로 채울 수 있습니다.
        #padding=True ,
        add_special_tokens=True
        )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask

In [ ]:
#sample_dataset = Dataset_finance_emoton_csv("") #모델 테스트용
sample_dataset = Dataset_samsung_news("")
sample_dataset.dataset

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,일자,언론사,제목,본문
0,20181231,한국일보,"화웨이, 글로벌 판매량 힘입어 프리미엄 스마트폰 첫 10%대 점유율",중국 화웨이가 3분기 프리미엄 스마트폰 시장에서 처음으로 10% 이상 점유율을 기록...
1,20181231,한국일보,증시 주저앉으며... 주식 100대 부자 자산 25조 증발,2018년 국내 증시가 주저 앉으면서 상장 주식 100대 부자의 자산이 한해 동안 ...
2,20181231,한국경제,"5G 보급 본격화, 헬스케어도 高성장 클라우드 엔터 자율車 바이오 '유망'",도전 2019 - 주식 시장 \n \n성장세 뚜렷한 테마는 \n \n \n[ 오형주...
3,20181231,서울경제,여명-눈물 닦아주는 기해년을 기대하며,홍준석 증권부장 \n \n연기자가 울고 예능인도 운다. 주말 내내 이어진 눈물의 시...
4,20181231,서울경제,2019 증시 키워드 GOLD PIG,지난해 천당과 지옥을 오갔던 국내 주식시장이 기해년 새해를 맞아 힘찬 도약을 준비하...
...,...,...,...,...
70332,20220101,매일경제,"""반도체부터 리츠까지"" 증권사가 '이구동성' 추천한 주식은?",올해 증권시장에서 코스피와 코스닥이 박스권으로 한 해의 거래를 마감해 아쉬움을 남기...
70333,20220101,세계일보,주식부호 톱 5위에 ‘이재용 홍라희 이부진 이서현’ 올라,"지난해 국내 증시 시가총액이 2020년에 비해 283조원 가까이 늘어난 가운데, 주..."
70334,20220101,서울경제,연말 효과 무색 코스피 '1월 효과'로 다시 날까 [다음주 증시전망],12월의 마지막 주 ‘연말 랠리’를 바라봤던 투자자들의 심경이 무색하게도 코스피는 ...
70335,20220101,한국경제,"""올 한국증시, 미국보다 나을 수도"" 펀드매니저들 입 모았다 [다시 열자! 3000시대]","지겨운 ‘코로나19’라는 단어에서 조금이라도 벗어나길 바랐지만, 여전히 주식시장에 ..."


### 모델 생성 : snunlp/KR-FinBert-SC

In [ ]:
# 모델 생성 : snunlp/KR-FinBert-SC
# https://huggingface.co/snunlp/KR-FinBert-SC
# https://github.com/snunlp/KR-FinBert-SC

model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC",num_labels=3).to(device)
#model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert",num_labels=3)
#model.cuda()

### 데이터 로딩 및 배치사이즈 정의

In [ ]:
# 데이터 로딩
batchSize = 30 # GPU 메모리 부족 오류나면 수치를 줄일것

sample_loader = DataLoader(sample_dataset, batch_size=batchSize, shuffle=False)

sample_result = sample_dataset.dataset.copy(deep=True)
print(sample_result)

             일자   언론사                                                 제목  \
0      20181231  한국일보              화웨이, 글로벌 판매량 힘입어 프리미엄 스마트폰 첫 10%대 점유율   
1      20181231  한국일보                   증시 주저앉으며... 주식 100대 부자 자산 25조 증발   
2      20181231  한국경제          5G 보급 본격화, 헬스케어도 高성장 클라우드 엔터 자율車 바이오 '유망'   
3      20181231  서울경제                               여명-눈물 닦아주는 기해년을 기대하며   
4      20181231  서울경제                               2019 증시 키워드 GOLD PIG   
...         ...   ...                                                ...   
70332  20220101  매일경제                  "반도체부터 리츠까지" 증권사가 '이구동성' 추천한 주식은?   
70333  20220101  세계일보                    주식부호 톱 5위에 ‘이재용 홍라희 이부진 이서현’ 올라   
70334  20220101  서울경제             연말 효과 무색 코스피 '1월 효과'로 다시 날까 [다음주 증시전망]   
70335  20220101  한국경제  "올 한국증시, 미국보다 나을 수도" 펀드매니저들 입 모았다 [다시 열자! 3000시대]   
70336  20220101  아주경제                     삼성 LG전자, 새해 경영 좌표는 "혁신적인 고객경험"   

                                                      본문  
0      중국 화웨이가 3분기 프리미엄 스마트폰

### 모델 실행 및 감성예측

In [ ]:
model.eval() #모델 평가모드로 변경

# GPU 캐쉬 비워줌
gc.collect()
torch.cuda.empty_cache()

idx_s = 0
idx_e = 0

#label_dict = {0: 'negative', 1: 'neutral', 2: 'positive'}
print(model.config.id2label) # 모델의 분류 종류 보기

#예측 결과 초기화
sample_result['pred_label'] = 0 # 예측 분류값
sample_result['negative'] = 0 # 부정적 감성 확률
sample_result['neutral'] = 0 # 보통 확률
sample_result['positive'] = 0 # 긍정적 감성 확률

# 배치 사이즈별 예측 실행
# tqdm은 진행상황을 표시해줌
#for input_ids_batch, attention_masks_batch,y_batch in tqdm(sample_loader):
for input_ids_batch, attention_masks_batch in tqdm(sample_loader):
  #y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1) #예측한 원소중 max값 1개만 반환
  predictions = torch.nn.functional.softmax(y_pred, dim=-1) # 예측한 원소의 값이 0과 1사이의 값으로 변환 (원소 합은 1)
  #print(predictions)
  
  rsList = list(map(int, predicted)) # 결과를 한번에 저장하기 위해 LIST로 변환 처리
  rs_negative = predictions[:, 0].tolist() #부정
  rs_neutral = predictions[:, 1].tolist() #보통
  rs_positive = predictions[:, 2].tolist() #긍정

  print(rsList)
  idx_e += len(rsList) #해당 배치구간내에 index 끝값 계산
  idx_s = idx_e - len(rsList)
  print("index==", idx_s, idx_e)

  sample_result['pred_label'][idx_s : idx_e] = rsList #배치사이즈 만큼 예측된 값 일괄 저장
  sample_result['negative'][idx_s : idx_e] = rs_negative
  sample_result['neutral'][idx_s : idx_e] = rs_neutral
  sample_result['positive'][idx_s : idx_e] = rs_positive


{0: 'negative', 1: 'neutral', 2: 'positive'}


  0%|          | 0/2345 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

[2, 1, 2, 1, 1, 1, 1, 0, 0, 2, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 2, 1]
index== 0 30


  0%|          | 2/2345 [00:00<15:20,  2.54it/s]

[1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0]
index== 30 60


...

 16%|█▌        | 374/2345 [02:38<14:28,  2.27it/s]

[0, 1, 1, 0, 2, 0, 1, 2, 1, 2, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 2, 1, 0]
index== 11190 11220


 16%|█▌        | 375/2345 [02:38<14:29,  2.27it/s]

[1, 1, 2, 1, 0, 1, 1, 1, 2, 1, 1, 2, 2, 0, 2, 2, 1, 1, 1, 2, 1, 2, 1, 0, 2, 1, 1, 1, 1, 1]
index== 11220 11250


 16%|█▌        | 376/2345 [02:39<14:26,  2.27it/s]

In [ ]:
print(sample_result)

### 예측 결과를 파일로 저장

In [ ]:
# 예측 결과를 파일로 저장
#sample_csv = sample_result.to_csv('sample.csv')
sample_xlsx = sample_result.to_excel('samsung_news_emotion_20180101-20220607_title.xlsx',index=False)
sample_csv = sample_result.to_csv('samsung_news_emotion_20180101-20220607_title.csv',sep=',',na_rep='NaN', columns=['일자','언론사','제목','본문','pred_label','negative','neutral','positive'],index=False)

#sample_xlsx = sample_result.to_excel('samsung_news_emotion_20180101-20220607_contents.xlsx',index=False)
#sample_csv = sample_result.to_csv('samsung_news_emotion_20180101-20220607_contents.csv',sep=',',na_rep='NaN', columns=['일자','언론사','제목','본문','pred_label','negative','neutral','positive'],index=False)


### 파일을 PC로 다운로드 하기

In [ ]:
# 파일을 PC로 다운로드 하기
from google.colab import files
#files.download('sample_result.csv')
files.download('samsung_news_emotion_20180101-20220607_title.xlsx')
files.download('samsung_news_emotion_20180101-20220607_title.csv')

#files.download('samsung_news_emotion_20180101-20220607_contents.xlsx')
#files.download('samsung_news_emotion_20180101-20220607_contents.csv')

In [ ]:
pd.read_excel("samsung_news_emotion_20180101-20220607_title.xlsx")

In [ ]:
sample_result.groupby("일자").mean()

,pred_label,negative,neutral,positive
일자,,,,
20180101,1.000000,0.017331,0.973516,0.009153
20180102,1.150000,0.058584,0.726884,0.214532
20180103,1.209877,0.152805,0.520842,0.326353
20180104,0.982456,0.170499,0.685041,0.144460
20180105,1.240741,0.076499,0.616316,0.307186
...,...,...,...,...
20220603,1.028571,0.148613,0.676578,0.174809
20220604,0.666667,0.486890,0.506659,0.006451
20220605,1.000000,0.230194,0.587380,0.182426


In [ ]:
sample_result.groupby("일자").sum()

,pred_label,negative,neutral,positive
일자,,,,
20180101,18,0.311959,17.523282,0.164759
20180102,92,4.686755,58.150689,17.162556
20180103,98,12.377176,42.188215,26.434609
20180104,56,9.718460,39.047334,8.234207
20180105,67,4.130927,33.281044,16.588030
...,...,...,...,...
20220603,36,5.201451,23.680241,6.118307
20220604,2,1.460670,1.519976,0.019353
20220605,10,2.301941,5.873804,1.824255
